# Encrypted Computations Trip and Tricks
## FHE setup


In [1]:
!git clone https://github.com/Huelse/SEAL-Python
!cd SEAL-Python/SEAL/native/src;cmake .;make
!pip3 install -r SEAL-Python/requirements.txt
!cd SEAL-Python;python3 setup.py install

Cloning into 'SEAL-Python'...
remote: Enumerating objects: 1115, done.
remote: Total 1115 (delta 0), reused 0 (delta 0), pack-reused 1115
Receiving objects: 100% (1115/1115), 8.47 MiB | 12.41 MiB/s, done.
Resolving deltas: 100% (557/557), done.
-- The CXX compiler identification is GNU 7.5.0
-- The C compiler identification is GNU 7.5.0
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Build type (CMAKE_BUILD_TYPE): Release
-- Microsoft SEAL debug mode: OFF
-- Library build type (SEAL_LIB_BUILD_TYPE): Static_PIC
-- Looking for C++ include x8

## 1. Basic Usage

In [36]:
from seal import *
from datetime import datetime
start=datetime.now()

# init params
parms = EncryptionParameters(scheme_type.BFV)

# Larger poly_modulus_degree makes ciphertext sizes larger and all operations
#    slower, but enables more complicated encrypted computations. Recommended
#    values are 1024, 2048, 4096, 8192, 16384, 32768, but it is also possible
#    to go beyond this range.

poly_modulus_degree = 4096
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(CoeffModulus.BFVDefault(poly_modulus_degree))

# The plaintext modulus can be any positive integer
# The plaintext
# modulus determines the size of the plaintext data type and the consumption
# of noise budget in multiplications. Thus, it is essential to try to keep the
# plaintext data type as small as possible for best performance. 
#
# (!) Please note that each encrypted value has a noise budget. Noise budget is getting consumed 
# during operations on cyphertexts. Once the noise budget of a 
# ciphertext reaches zero it becomes too corrupted to be decrypted.
parms.set_plain_modulus(256)
context = SEALContext.Create(parms)

keygen = KeyGenerator(context)
public_key = keygen.public_key()
secret_key = keygen.secret_key()

encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)
decryptor = Decryptor(context, secret_key)

# 2. Encrypted operations

In [37]:
start=datetime.now()

enc1 = Ciphertext()
enc2 = Ciphertext()
enc3 = Ciphertext()

# hex numbers
encryptor.encrypt(Plaintext("A1"), enc1)
encryptor.encrypt(Plaintext("2E"), enc2)
encryptor.encrypt(Plaintext("3E"), enc3)
evaluator.add_inplace(enc1, enc2)
evaluator.multiply_inplace(enc3, enc2)

# decrypt
result_add = Plaintext()
result_mul = Plaintext()
decryptor.decrypt(enc1, result_add)
decryptor.decrypt(enc3, result_mul)
# 0xA1 + 0x2E = 0xCF
# 0x2E * 0x3E = 0xB24
# remember that all the computation perform within the given modulus (set_plain_modulus which is 256 = 0x100)
# so the correct values are 
# (0xA1 + 0x2E) mod 0x100 = 0xCF
# (0x2E * 0x3E) mod 0x100 = 0xB24
print(f"result_add: 0x{result_add.to_string()}, result_mul 0x{result_mul.to_string()}")

print(f"completed in {datetime.now()-start}")

result_add: 0xCF, result_mul 0x24
completed in 0:00:00.022250


# 3. Relinearization
This is an operation that reduces the size of a ciphertext after multiplication back to the initial size

In [43]:
# there is a special key for it
relin_keys = keygen.relin_keys()
print("size of enc3 value before Relinearization: " + str(enc3.size()))
evaluator.relinearize_inplace(enc3, relin_keys)
print("size of enc3 value after Relinearization: " + str(enc3.size()))


size of enc3 value before Relinearization: 3
size of enc3 value after Relinearization: 2


# 4. You can apply an operation on two large arrays in parallel. This is called batching
This is "recomended" way of most computations

In [24]:
from seal import *
from datetime import datetime
start=datetime.now()

# init fhe
parms = EncryptionParameters(scheme_type.BFV)
poly_modulus_degree = 8192
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(CoeffModulus.BFVDefault(poly_modulus_degree))

# To enable batching, we need to set the plain_modulus to be a prime number
# congruent to 1 modulo 2*poly_modulus_degree. Microsoft SEAL provides a helper
# method for finding such a prime. In this example we create a 20-bit prime
# that supports batching.
parms.set_plain_modulus(PlainModulus.Batching(poly_modulus_degree, 20))
context = SEALContext.Create(parms)

# key generation
keygen = KeyGenerator(context)
public_key = keygen.public_key()
secret_key = keygen.secret_key()
encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)
decryptor = Decryptor(context, secret_key)

# create batch enoder
batch_encoder = BatchEncoder(context)
slot_count = batch_encoder.slot_count()

# each  vector is a constant size of the poly_modulus_degree value (8192)
# set the values you want in each cell

#encrypt vec #1 ( each number is 7 )
plain_vec1 = uIntVector( [7] * slot_count )
plain_text1 = Plaintext()
# encode python matrix in Plaintext
batch_encoder.encode(plain_vec1, plain_text1)
# and encrypt
encrypted1 = Ciphertext()
encryptor.encrypt(plain_text1, encrypted1)

#encrypt vec #2 (  each number is 3 )
plain_vec2 = uIntVector( [3] * slot_count )
plain_text2 = Plaintext()
batch_encoder.encode(plain_vec2, plain_text2)
encrypted2 = Ciphertext()
encryptor.encrypt(plain_text2, encrypted2)

#  multiply encrypted vectors in parallel
evaluator.multiply_inplace(encrypted1, encrypted2);

# decrypt
plain_result = Plaintext()
decryptor.decrypt(encrypted1, plain_result);

pod_result = uIntVector()
batch_encoder.decode(plain_result, pod_result);

# The result should be: 8192 x 7 x 3 = 172032
print( f"{pod_result[:3]}...f{pod_result[-3:]} sum: {sum(pod_result)} ")

print(f"completed in {datetime.now()-start}")

uIntVector[21, 21, 21]...fuIntVector[21, 21, 21] sum: 172032 
completed in 0:00:00.112214
